In [1]:
# all the imports you should need for this notebook
import pandas as pd
from datetime import datetime

In [10]:
# This only needs to be run ONCE to fix the broken datetimes in the hail and tornado data.
# It will create new files with the fixed datetimes and a combined file for sharing.
# If you run this again, it will overwrite the existing files, so be careful!

# the combined file is uploaded to the google bucket in the ExtremeWeatherBench project
# and can be accessed at gs://extremeweatherbench/datasets/AustralianLSRData_2020-2024.csv

def parse_datetime(row):
    """
    Parses date and time components from a DataFrame row into a single datetime object.
    
    Args:
        row: A pandas Series representing a row of the DataFrame.

    Returns:
        A datetime object.
    """
    # Assuming a fixed year for simplicity, adjust as needed.
    padded_day = str(row['Day']).zfill(2)
    padded_month = str(row['Month']).zfill(2)
    padded_time = str(row['Time (UTC)']).zfill(4)
    date_string = f"{padded_month}/{padded_day}/{row['Year']} {padded_time}"
    #print(f"Parsing date string: {date_string}")  # Debugging line
    return datetime.strptime(date_string, '%m/%d/%Y %H%M')


# fix the broken datetimes in the australian hail data
hail_file = "/home/amy/NHP_edited.csv"
df_hail = pd.read_csv(hail_file, delimiter=',', engine='python', parse_dates=['Date & Time of Observation'], infer_datetime_format=True)

# rename the date column to match the tornado data
df_hail.rename(columns={'x': 'Longitude', 'y': 'Latitude'}, inplace=True)
df_hail.rename(columns={'Date & Time of Observation': 'Date/Time UTC'}, inplace=True)
df_hail = df_hail[['Date/Time UTC', 'Latitude', 'Longitude', 'Event Name', 'Minimum Hail Dimension (mm)', "Maximum Hail Dimension (mm)"]]

# Crate a report_type column
df_hail['report_type'] = 'hail'

# save the new file
new_hail_file = '/home/amy/NHP_2020-2024_cleaned_fixed.csv'
df_hail.to_csv(new_hail_file, index=False)

# fix the broken date-times in the tornado data
tor_file = "/home/amy/NTP_edited2.csv"

# parse the dates from the Day	Month	Time (UTC) columns
df_tor = pd.read_csv(tor_file, delimiter=',', engine='python')
# Apply the function to create a new 'Datetime' column
df_tor['Date/Time UTC'] = df_tor.apply(parse_datetime, axis=1)
df_tor.rename(columns={'x': 'Longitude', 'y': 'Latitude'}, inplace=True)

df_tor = df_tor[['Date/Time UTC', 'Latitude', 'Longitude', 'Event Name', 'Event Type', "Damage"]]

# subselect only the torando events on the ground (no waterspouts etc)
df_tor = df_tor[df_tor['Event Type'] == 'tornado_over_land']

# Crate a report_type column
df_tor['report_type'] = 'tor'

# save the new file
new_tor_file = '/home/amy/CanadaTor2020-2024_cleaned_fixed.csv'
df_tor.to_csv(new_tor_file, index=False)

# create a combined file for sharing
df_combined = pd.concat([df_hail, df_tor], ignore_index=True)
combined_file = '/home/amy/CanadaLSRData_2020-2024.csv'

# save the combined file
df_combined.to_csv(combined_file, index=False)

print ("DONE")
#print the types of the dataframe
print(df_combined.dtypes)

DONE
Date/Time UTC                  datetime64[ns]
Latitude                              float64
Longitude                             float64
Event Name                             object
Minimum Hail Dimension (mm)           float64
Maximum Hail Dimension (mm)           float64
report_type                            object
Event Type                             object
Damage                                 object
dtype: object


/tmp/ipykernel_2440/2335348170.py:29: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_hail = pd.read_csv(hail_file, delimiter=',', engine='python', parse_dates=['Date & Time of Observation'], infer_datetime_format=True)
/tmp/ipykernel_2440/2335348170.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_hail = pd.read_csv(hail_file, delimiter=',', engine='python', parse_dates=['Date & Time of Observation'], infer_datetime_format=True)
